# Assignment 1 - Production Technology

The dataset contains `N = 441` firms observed over `T = 12` years, 1968-1979. There variables are: 
* `lcap`: Log of capital stock, $k_{it}$ 
* `lemp`: log of employment, $\ell_{it}$ 
* `ldsa`: log of deflated sales, $y_{it}$
* `year`: the calendar year of the observation, `year` $ = 1968, ..., 1979$, 
* `firmid`: anonymized indicator variable for the firm, $i = 1, ..., N$, with $N=441$.

In [42]:
import numpy as np
from numpy import linalg as la
import pandas as pd
from io import StringIO
from tabulate import tabulate
from matplotlib import pyplot as plt
from scipy.stats import chi2
from scipy.stats import t
import seaborn as sns
import matplotlib.pyplot as plt
from numpy.random import default_rng

# Import this weeks LinearModels .py file
import LinearModels_assign as lm
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
dat = pd.read_csv('firms.csv')

## Data

In [44]:
dat_array = np.array(dat)

id_array = dat_array[:, 0]

# Count how many persons we have. This returns a tuple with the unique IDs,
# and the number of times each person is observed.
unique_id = np.unique(id_array, return_counts=True)
n = unique_id[0].size
t = int(unique_id[1].mean())
year = np.array(dat_array[:, 1], dtype=int)

# Load the rest of the data into arrays.
y = dat_array[:,4].reshape(-1,1)
x = np.column_stack(( np.ones(y.shape[0]), dat_array[:,2:4]))

# Labels
# Lets also make some variable names
label_y = 'ldsa'
label_x = [
    'Constant', 
    'lcap', 
    'lemp', 
]

## Estimating the model with pooled OLS

In [45]:
ols_result = lm.estimate(y, x, robust_se=True, t=t, n=n)
lm.print_table(
    (label_y, label_x), ols_result, title="Pooled OLS", floatfmt='.4f'
)

Pooled OLS
Dependent variable: ldsa

            Beta      Se    t-values
--------  ------  ------  ----------
Constant  0.0000  0.0161      0.0000
lcap      0.3100  0.0324      9.5810
lemp      0.6748  0.0366     18.4526
R² = 0.914
σ² = 0.131


## Estimating a fixed effects (FE) model

#### Demeaning data

In [46]:
def demeaning_matrix(t):
    Q_T = np.eye(t) - np.tile(1/t, (t, t))
    return Q_T

In [47]:
Q_T = demeaning_matrix(t)
y_demean = lm.perm(Q_T, y)
x_demean = lm.perm(Q_T, x)

#### Checking the rank

In [48]:
# Check rank of demeaned matrix, and return its eigenvalues.
def check_rank(x):
    print(f'Rank of x: {la.matrix_rank(x)}')
    lambdas, V = la.eig(x.T@x)
    np.set_printoptions(suppress=True)  # This is just to print nicely.
    print(f'Eigenvalues of within-transformed x: {lambdas.round(decimals=0)}')
check_rank(x_demean)

Rank of x: 2
Eigenvalues of within-transformed x: [  0.  58. 214.]


The constant is constant over time and therefore has to be removed before using the FE estimator. (x has to be of full rank)

In [49]:
x_demean = x_demean[:,1:]
label_x_fe = label_x[1:]

#### Estimating the FE model

In [50]:
# Estimate FE OLS using the demeaned variables.
fe_result = lm.estimate(
    y_demean, x_demean, transform='fe', robust_se=True, t=t, n=n
)
lm.print_table(
    (label_y, label_x_fe), 
    fe_result, title='FE regression', floatfmt='.4f'
)

FE regression
Dependent variable: ldsa

        Beta      Se    t-values
----  ------  ------  ----------
lcap  0.1546  0.0299      5.1630
lemp  0.6942  0.0417     16.6674
R² = 0.477
σ² = 0.018


## Estimating a first differences (FD) model

#### Transforming data, taking first differences of y and x

In [10]:
def fd_matrix(t):
    D_T = np.eye(t) - np.eye(t, k=-1)
    D_T = D_T[1:]
    return D_T

In [11]:
# Transform the data.
D_T = fd_matrix(t)
y_diff = lm.perm(D_T, y)
x_diff = lm.perm(D_T, x)

#### Checking the rank

In [12]:
check_rank(x_diff)

Rank of x: 2
Eigenvalues of within-transformed x: [47. 35.  0.]


In [13]:
# we remove the constant as our x_diff is not of full rank
x_diff = x_diff[:,1:]

#### Estimating the FD model

In [14]:
fd_result = lm.estimate(
    y_diff, x_diff, robust_se=True, t=t, n=n
)
lm.print_table(
    (label_y, label_x_fe), 
    fd_result, title='FD regression', floatfmt='.4f'
)

FD regression
Dependent variable: ldsa

        Beta      Se    t-values
----  ------  ------  ----------
lcap  0.0630  0.0191      3.3043
lemp  0.5487  0.0183     29.9635
R² = 0.165
σ² = 0.014


## Estimating a random effects (RE) model

#### Quasi-demeaning the variables

First we create the mean matrix $P_t$

In [15]:
def mean_matrix(t):
    P_T = np.tile(1/t, (t, t))
    return P_T

In [16]:
# Transform the data.
P_T = mean_matrix(t)
y_mean = lm.perm(P_T, y)
x_mean = lm.perm(P_T, x)

#### Check rank of x

In [17]:
check_rank(x_mean)

Rank of x: 3
Eigenvalues of within-transformed x: [ 5292. 15515.   675.]


Matrix is of full rank

#### We now estimate using the between estimator

In [18]:
be_result = lm.estimate(y_mean, x_mean, robust_se=False, t=t, n=n)
lm.print_table(
    (label_y, label_x), be_result, title="Pooled OLS", floatfmt='.4f'
)

Pooled OLS
Dependent variable: ldsa

            Beta      Se    t-values
--------  ------  ------  ----------
Constant  0.0000  0.0046      0.0000
lcap      0.3188  0.0089     35.8720
lemp      0.6672  0.0099     67.6130
R² = 0.923
σ² = 0.114


#### We now estimate $\hat \lambda$ using $\hat{\sigma}_u ^2$ and $\hat{\sigma}_c ^2$:

$$\hat{\lambda} = 1 - \sqrt{\frac{\widehat{\sigma}_{u}^{2}}{(\widehat{\sigma}_{u}^{2} + T\widehat{\sigma}_{c}^{2})}}, $$

In [19]:
sigma_FE = fe_result["sigma"][0][0]
sigma_BE = be_result["sigma"][0][0] - sigma_FE/t

In [20]:
lamb = 1 - np.sqrt(sigma_FE/(sigma_FE + t*sigma_BE))

#### Using $\lambda$ we now quasi demean our variables

In [21]:
def quasi_matrix(t):
    Q_T = np.eye(t) - np.tile(lamb*1/t, (t, t))
    return Q_T

In [22]:
Q_t = quasi_matrix(t)
y_quasi = lm.perm(Q_t, y)
x_quasi = lm.perm(Q_t, x)

#### Check rank

In [23]:
check_rank(x_quasi)

Rank of x: 3
Eigenvalues of within-transformed x: [ 68. 412.  67.]


#### Estimating quasi demeaned variables using POLS

In [24]:
re_result = lm.estimate(y_quasi, x_quasi, transform = "re", robust_se=False, t=t, n=n)
lm.print_table(
    (label_y, label_x), re_result, title="Pooled OLS", floatfmt='.4f'
)

Pooled OLS
Dependent variable: ldsa

            Beta      Se    t-values
--------  ------  ------  ----------
Constant  0.0000  0.0162      0.0000
lcap      0.1990  0.0117     17.0431
lemp      0.7197  0.0131     54.8444
R² = 0.643
σ² = 0.018


#### Hausman test

In [25]:
# FILL IN
# Follow the steps in the question
# Calculate without Robust SE!
hat_diff =  fe_result['b_hat'] - re_result["b_hat"][1:]
cov_diff =  fe_result['cov'] - re_result['cov'][1:,1:]
H =  hat_diff.T @ la.inv(cov_diff) @ hat_diff

# This calculates the p-value of the Hausman test.
p_val = chi2.sf(H.item(), 2)

In [26]:
# This code takes the results that you have made, and prints a nice looking table.
def print_h_test(fe_result, re_result, hat_diff, p_val):
    table = []
    for i in range(len(hat_diff)):
        row = [
            fe_result['b_hat'][i], re_result['b_hat'][1:][i], hat_diff[i]
        ]
        table.append(row)

    print(tabulate(
        table, headers=['b_fe', 'b_re', 'b_diff'], floatfmt='.4f'
        ))
    print(f'The Hausman test statistic is: {H.item():.2f}, with p-value: {p_val:.2f}.')
print_h_test(fe_result, re_result, hat_diff, p_val)

  b_fe    b_re    b_diff
------  ------  --------
0.1546  0.1990   -0.0444
0.6942  0.7197   -0.0255
The Hausman test statistic is: 73.87, with p-value: 0.00.


Conducting the Hausman test using wald statistic implies that RE.3 holds. This might not be the case. Therefore, we use bootstrap, thereby not relying on assumptions on homoscedasticity. 

In [27]:
# Set up function for sub samples
def hat_cov_diff(x,y,t,n):
    
    # estimate FE and RE regression on subsample
    N,K = x.shape
    
    # FE
    # demean
    Q_T_i = demeaning_matrix(t)
    y_demean_i = lm.perm(Q_T, y)
    x_demean_i = lm.perm(Q_T, x)
    #rank and remove if not full rank
    x_demean_i = x_demean_i[:,K-la.matrix_rank(x_demean_i):]
    # estimate
    fe_result_i = lm.estimate(
    y_demean_i, x_demean_i, transform='fe', t=t, n=n
    )
    
    # RE
    # between estimator
    P_T_i = mean_matrix(t)
    y_mean_i = lm.perm(P_T_i, y)
    x_mean_i = lm.perm(P_T_i, x)
    # rank
    x_demean_i = x_mean_i[:,K-la.matrix_rank(x_mean_i):]
    be_result_i = lm.estimate(y_mean_i, x_mean_i, t=t, n=n)
    #Lambda
    sigma_FE_i = fe_result_i["sigma"][0][0]
    sigma_BE_i = be_result_i["sigma"][0][0] - sigma_FE/t
    lamb = 1 - np.sqrt(sigma_FE_i/(sigma_FE_i + t*sigma_BE_i))
    # quasi demean
    Q_t_i = quasi_matrix(t)
    y_quasi_i = lm.perm(Q_t_i, y)
    x_quasi_i = lm.perm(Q_t_i, x)
    # rank
    x_quasi_i = x_quasi_i[:,K-la.matrix_rank(x_mean_i):]
    re_result_i = lm.estimate(y_quasi_i, x_quasi_i, transform = "re", t=t, n=n)
    
    # Estimate Hausman statistic
    hat_diff_lcap =  fe_result_i['b_hat'][0] - re_result_i["b_hat"][1]
    hat_diff_lemp =  fe_result_i['b_hat'][1] - re_result_i["b_hat"][2]
    hatt_diff = fe_result_i['b_hat'] - re_result_i["b_hat"][1:]
    
    return hat_diff_lcap, hat_diff_lemp, hatt_diff
    

In [28]:
# Function for drawing bootsrap sample
def bootstrap_sample(dat,n):
    ii_boot = np.random.choice(range(1,n), n, replace=True)
    new_df = dat.loc[dat['firmid'].isin(ii_boot)]
    
    # Set up function for sub samples

    # We need to know the number of 
    dat_array = np.array(new_df)

    id_array = dat_array[:, 0]

    # Count how many persons we have. This returns a tuple with the unique IDs,
    # and the number of times each person is observed.
    unique_id = np.unique(id_array, return_counts=True)
    n_i = unique_id[0].size
    t_i = int(unique_id[1].mean())
    year_i = np.array(dat_array[:, 1], dtype=int)

    # Load the rest of the data into arrays.
    y_i = dat_array[:,4].reshape(-1,1)
    x_i = np.column_stack(( np.ones(y_i.shape[0]), dat_array[:,2:4]))
    
    return y_i, x_i, n_i, t_i, year_i

In [29]:
# Run bootsrap procedure
nboot = 1000  # Number of bootstraps, should ideally be very large 

# Set seed for random sampling.
seed = 42
rng = default_rng()

# initialize 
hat_diff_o = np.empty((nboot,2,1))
hat_diff_lcap_o = np.empty((nboot,1))
hat_diff_lemp_o = np.empty((nboot,1))

for i in range(nboot): 
    print(f'Bootstrap iteration {i+1}/{nboot}')
    
    # 1. draw sample from data 
    y_i, x_i, n_i, t_i, year_i = bootstrap_sample(dat,n)
    
    # 2. estimate and compute 
    hat_diff_lcap_0, hat_diff_lemp_0, hat_diff_0 = hat_cov_diff(x_i,y_i,t_i,n_i)
    
    # save for each iteration
    hat_diff_o [i] = hat_diff_0
    hat_diff_lcap_o [i] = hat_diff_lcap_0
    hat_diff_lemp_o [i] = hat_diff_lemp_0
    

Bootstrap iteration 1/1000
Bootstrap iteration 2/1000
Bootstrap iteration 3/1000
Bootstrap iteration 4/1000
Bootstrap iteration 5/1000
Bootstrap iteration 6/1000
Bootstrap iteration 7/1000
Bootstrap iteration 8/1000
Bootstrap iteration 9/1000
Bootstrap iteration 10/1000
Bootstrap iteration 11/1000
Bootstrap iteration 12/1000
Bootstrap iteration 13/1000
Bootstrap iteration 14/1000
Bootstrap iteration 15/1000
Bootstrap iteration 16/1000
Bootstrap iteration 17/1000
Bootstrap iteration 18/1000
Bootstrap iteration 19/1000
Bootstrap iteration 20/1000
Bootstrap iteration 21/1000
Bootstrap iteration 22/1000
Bootstrap iteration 23/1000
Bootstrap iteration 24/1000
Bootstrap iteration 25/1000
Bootstrap iteration 26/1000
Bootstrap iteration 27/1000
Bootstrap iteration 28/1000
Bootstrap iteration 29/1000
Bootstrap iteration 30/1000
Bootstrap iteration 31/1000
Bootstrap iteration 32/1000
Bootstrap iteration 33/1000
Bootstrap iteration 34/1000
Bootstrap iteration 35/1000
Bootstrap iteration 36/1000
B

KeyboardInterrupt: 

In [161]:
hat_diff_boot = hat_diff_o.reshape(2,nboot)

In [162]:
hat_diff_cov_boot = np.cov(hat_diff_boot)

In [165]:
hat_diff_cov_boot

array([[0.00013653, 0.00006335],
       [0.00006335, 0.00014711]])

In [163]:
# Calculate Hausman test using this statistic
H_boot =  hat_diff.T @ la.inv(hat_diff_cov_boot) @ hat_diff

In [164]:
H_boot

array([[14.65699423]])

In [170]:
chi2.sf(H_boot.item(), 2)

0.0006565595783555797

# Test for serial correlation

In [30]:
tt = int(x_diff.shape[0]/441)

In [31]:
# From exercises
def serial_corr(y, x, year):
    b_hat = lm.est_ols(y, x)
    e = y - x@b_hat
    
    Lag_T = np.eye(tt-1,tt, k=0)
    Lead_T = np.eye(tt-1,tt, k=1)
    
    e_l = lm.perm(Lag_T, e)
    e = lm.perm(Lead_T,e)
    
    return lm.estimate(e_l, e)

In [32]:
corr_result = serial_corr(y_diff, x_diff, tt)
label_ye = 'OLS residual, e\u1d62\u209c'
label_e = ['e\u1d62\u209c\u208B\u2081']
title = 'Serial Correlation'

lm.print_table(
    (label_ye, label_e), corr_result,
    title='Serial Correlation', floatfmt='.4f'
)


Serial Correlation
Dependent variable: OLS residual, eᵢₜ

          Beta      Se    t-values
-----  -------  ------  ----------
eᵢₜ₋₁  -0.1984  0.0148    -13.4493
R² = 0.039
σ² = 0.014


#### Test for strict exogeneity

In [33]:
# Create lead
F_T = np.eye(t, k=1)
F_T = F_T[:-1]

# Lead Capital and Labour (employment)
lcap_lead = lm.perm(F_T, x[:, 1].reshape(-1, 1))
lemp_lead = lm.perm(F_T, x[:, 2].reshape(-1, 1))

# Collect variables to test for exogeneity. We lead so we lose first observation
x_exo = x[year != 1979]
x_exo = np.hstack((x_exo, lcap_lead,lemp_lead))
y_exo = y[year != 1979]

# Make within transformation
Q_T_exo = demeaning_matrix(t-1)
y_demean_exo = lm.perm(Q_T_exo, y_exo)
x_demean_exo = lm.perm(Q_T_exo, x_exo)
x_demean_exo = x_demean_exo[:,1:]
label_x_fe_exo = label_x[1:] + ['Capital lead'] + ['Employement lead']

# Estimate FE model with the leaded variables
fe_result_exo = lm.estimate(
    y_demean_exo, x_demean_exo, transform='fe', robust_se=True, t=t, n=n
)

# Print table
lm.print_table(
    (label_y, label_x_fe_exo), 
    fe_result_exo, title='Strict exogeneity test', floatfmt='.4f'
)


Strict exogeneity test
Dependent variable: ldsa

                    Beta      Se    t-values
----------------  ------  ------  ----------
lcap              0.0280  0.0348      0.8051
lemp              0.5408  0.0381     14.1866
Capital lead      0.1667  0.0424      3.9302
Employement lead  0.1419  0.0263      5.4049
R² = 0.478
σ² = 0.016


### Testing the hypothesis lcap + lemp is = 1

In [51]:
R = np.array([1,1])
r = 1

In [52]:
wald_test = (R @ fe_result['b_hat'] - r) / ( R @ fe_result['cov'] @ R.T) * (R @ fe_result['b_hat'] - r)

In [53]:
wald_test

array([19.40291252])

$\chi^2(1)$ distribution 

In [54]:
p_val_wald = chi2.sf(wald_test.item(), 1)

In [55]:
print(float(p_val_wald))

1.0584551670864466e-05
